# NeuroFetal AI - Enhanced Fusion ResNet Training

**Version 2.0** - Advanced Enhancements:
- 3-Input Architecture (FHR + Tabular + CSP)
- Squeeze-and-Excitation (SE) Blocks
- Multi-Head Self-Attention
- Focal Loss for Class Imbalance

### Instructions:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run cells in order
3. Use ** Update Repo** cell anytime to pull latest changes

## Step 0: GitHub Authentication

In [1]:
from getpass import getpass
import os

GITHUB_REPO = "Krishna200608/NeuroFetal-AI"
GITHUB_TOKEN = getpass("GitHub Personal Access Token: ")

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO
print("Token saved to session.")

GitHub Personal Access Token: ··········
Token saved to session.


## Step 1: Clone Repository

In [2]:
import os
import shutil

# Always start from /content
os.chdir("/content")

GITHUB_REPO = os.environ.get('GITHUB_REPO', 'Krishna200608/NeuroFetal-AI')
GITHUB_TOKEN = os.environ.get('GITHUB_TOKEN', '')

# Clean clone for fresh start
if os.path.exists("NeuroFetal-AI"):
    shutil.rmtree("NeuroFetal-AI")
    print("Removed old clone.")

print("Cloning repository...")
if GITHUB_TOKEN:
    !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git
else:
    !git clone https://github.com/{GITHUB_REPO}.git

# Verify
dataset_path = "/content/NeuroFetal-AI/Datasets/ctu_uhb_data"
if os.path.exists(dataset_path):
    count = len([f for f in os.listdir(dataset_path) if f.endswith('.dat')])
    print(f"\nSUCCESS: Found {count} dataset records!")
else:
    print(f"\n ERROR: Dataset folder not found!")

Cloning repository...
Cloning into 'NeuroFetal-AI'...
remote: Enumerating objects: 1451, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 1451 (delta 42), reused 45 (delta 14), pack-reused 1365 (from 2)
Receiving objects: 100% (1451/1451), 108.88 MiB | 32.69 MiB/s, done.
Resolving deltas: 100% (724/724), done.

SUCCESS: Found 552 dataset records!


## Update Repo (Run anytime to pull latest changes)
**Run this cell whenever you make changes on GitHub or encounter errors.**

In [27]:
import os
os.chdir("/content/NeuroFetal-AI")
# !git checkout -- Code/models/
!git pull origin main
print("\n Repository updated! Re-run training cells below.")

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 7 (delta 6), reused 7 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 1.44 KiB | 493.00 KiB/s, done.
From https://github.com/Krishna200608/NeuroFetal-AI
 * branch            main       -> FETCH_HEAD
   ea057b1..a17c71c  main       -> origin/main
Updating ea057b1..a17c71c
Fast-forward
 Code/scripts/train.py      |  4 ++--
 Code/utils/csp_features.py | 17 +++++++++++++++++
 2 files changed, 19 insertions(+), 2 deletions(-)

 Repository updated! Re-run training cells below.


## Step 2: Setup Paths

In [29]:
import os
import sys

PROJECT_ROOT = "/content/NeuroFetal-AI"
CODE_DIR = os.path.join(PROJECT_ROOT, "Code")
SCRIPTS_DIR = os.path.join(CODE_DIR, "scripts")
UTILS_DIR = os.path.join(CODE_DIR, "utils")

os.chdir(SCRIPTS_DIR)
sys.path.insert(0, SCRIPTS_DIR)
sys.path.insert(0, UTILS_DIR)
sys.path.insert(0, CODE_DIR)

print(f" Working dir: {os.getcwd()}")

 Working dir: /content/NeuroFetal-AI/Code/scripts


## Step 3: Install Dependencies

In [5]:
!pip install -q wfdb shap scipy --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 6.5 MB/s eta 0:00:00


In [6]:
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

TensorFlow: 2.19.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Step 4: Data Ingestion

In [23]:
# Cell: Remove old processed data (run once before re-ingestion)
import os

processed_dir = "/content/NeuroFetal-AI/Datasets/processed"
files_to_remove = ["X_fhr.npy", "X_tabular.npy", "y.npy", "X_uc.npy"]

for f in files_to_remove:
    path = os.path.join(processed_dir, f)
    if os.path.exists(path):
        os.remove(path)
        print(f"Removed: {f}")

print("Old data cleared. Ready for re-ingestion.")

Removed: X_fhr.npy
Removed: X_tabular.npy
Removed: y.npy
Removed: X_uc.npy
Old data cleared. Ready for re-ingestion.


In [24]:
import os
processed_dir = "/content/NeuroFetal-AI/Datasets/processed"

if os.path.exists(os.path.join(processed_dir, "X_fhr.npy")):
    print("Processed data exists. Skipping ingestion.")
else:
    print("Running data ingestion...")
    !python data_ingestion.py

Running data ingestion...
Found 552 records.
Processed 100 records...
Processed 200 records...
Processed 300 records...
Processed 400 records...
Processed 500 records...
Processing complete. Processed 552 patients into 2760 slices.
Shapes: X_fhr (2760, 1200), X_uc (2760, 1200), X_tabular (2760, 3), y (2760,)
Class balance: 200 compromised / 2760 total


In [25]:
# Add
import numpy as np

X_fhr = np.load(os.path.join(processed_dir, "X_fhr.npy"))
X_tabular = np.load(os.path.join(processed_dir, "X_tabular.npy"))
y = np.load(os.path.join(processed_dir, "y.npy"))

print(f"FHR: {X_fhr.shape}, Tabular: {X_tabular.shape}, Labels: {y.shape}")
print(f"Class Balance: {np.mean(y):.2%} positive")

FHR: (2760, 1200), Tabular: (2760, 3), Labels: (2760,)
Class Balance: 7.25% positive


## Step 5: Training

In [30]:
!python train.py

2026-02-05 06:56:38.066646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770274598.087075   32811 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770274598.093246   32811 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770274598.109969   32811 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770274598.109991   32811 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770274598.109997   32811 computation_placer.cc:177] computation placer alr

In [31]:
# 3. After training, run uncertainty evaluation
!python evaluate_uncertainty.py

2026-02-05 07:21:34.734757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770276094.754604   42238 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770276094.760680   42238 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770276094.775925   42238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770276094.775958   42238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770276094.775962   42238 computation_placer.cc:177] computation placer alr

## Step 6: Explainability (XAI)

In [11]:
!python xai.py

2026-02-05 05:28:03.618977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770269283.642990    5722 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770269283.653068    5722 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770269283.674615    5722 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770269283.674638    5722 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770269283.674644    5722 computation_placer.cc:177] computation placer alr

## Step 7: Push Results to GitHub

In [12]:
!git config --global user.email "krishnasikheriya001@gmail.com"
!git config --global user.name "Krishna200608"

In [13]:
import os
os.chdir("/content/NeuroFetal-AI")

!git add Code/models/*.keras Reports/training_logs/*.json Code/figures/*.png 2>/dev/null || true
!git status
!git commit -m "Add trained model from Colab" || echo "Nothing to commit"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Code/figures/shap_summary.png
	modified:   Code/models/enhanced_model_fold_1.keras
	modified:   Code/models/enhanced_model_fold_2.keras
	modified:   Code/models/enhanced_model_fold_3.keras
	modified:   Code/models/enhanced_model_fold_4.keras
	modified:   Code/models/enhanced_model_fold_5.keras
	new file:   Reports/training_logs/training_log_20260205_052759.json

[main 29f4972] Add trained model from Colab
 7 files changed, 49 insertions(+)
 rewrite Code/figures/shap_summary.png (83%)
 rewrite Code/models/enhanced_model_fold_1.keras (65%)
 rewrite Code/models/enhanced_model_fold_2.keras (66%)
 rewrite Code/models/enhanced_model_fold_3.keras (65%)
 rewrite Code/models/enhanced_model_fold_4.keras (65%)
 rewrite Code/models/enhanced_model_fold_5.keras (65%)
 create mode 100644 Reports/training_logs/training_log_20260205_052759.json
Enumerat

## Results

In [14]:
import json, glob, os

log_dir = "/content/NeuroFetal-AI/Reports/training_logs"
logs = sorted(glob.glob(os.path.join(log_dir, "training_log_*.json")))

if logs:
    with open(logs[-1]) as f:
        r = json.load(f)
    print(f"Mean AUC: {r['summary']['mean_auc']:.4f} ± {r['summary']['std_auc']:.4f}")
else:
    print("No training logs found.")

Mean AUC: 0.6482 ± 0.0396
